Import libraries

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
import folium
from folium import plugins
import shapely

Read data

In [2]:
df_demands = pd.read_csv("data/Demand_Mediterranean.csv", sep="\t")
df_ports = pd.read_csv("data/ports.csv", sep="\t")
df_dist = pd.read_csv("data/dist_dense.csv", sep="\t")
df_fleet = pd.read_csv("data/fleet_data.csv", sep="\t")
df_fleet_count = pd.read_csv("data/fleet_Mediterranean.csv", sep="\t")

df_ports = df_ports[df_ports["D_Region"] == "West Med"].dropna(how="any")

df_ports = df_ports.set_index("UNLocode")



df_demands["DestinationLat"] = df_demands["Destination"].map(df_ports["Latitude"])
df_demands["DestinationLong"] = df_demands["Destination"].map(df_ports["Longitude"])
df_demands["OriginLat"] = df_demands["Origin"].map(df_ports["Latitude"])
df_demands["OriginLong"] = df_demands["Origin"].map(df_ports["Longitude"])
df_demands["distance"] = df_demands.apply(lambda x: df_dist.loc[(df_dist["fromUNLOCODE"] == x["Origin"]) & (df_dist["toUNLOCODE"] == x["Destination"]), "Distance"].min(), axis=1)
df_demands["geometry"] = df_demands.apply(lambda x: shapely.geometry.LineString([[x["OriginLong"], x["OriginLat"]], [x["DestinationLong"], x["DestinationLat"]]]), axis=1)

df_dist = df_dist[df_dist["fromUNLOCODE"].isin(df_ports.index) & df_dist["toUNLOCODE"].isin(df_ports.index)]
df_dist["DestinationLat"] = df_dist["toUNLOCODE"].map(df_ports["Latitude"])
df_dist["DestinationLong"] = df_dist["toUNLOCODE"].map(df_ports["Longitude"])
df_dist["OriginLat"] = df_dist["fromUNLOCODE"].map(df_ports["Latitude"])
df_dist["OriginLong"] = df_dist["fromUNLOCODE"].map(df_ports["Longitude"])
df_dist["geometry"] = df_dist.apply(lambda x: shapely.geometry.LineString([[x["OriginLong"], x["OriginLat"]], [x["DestinationLong"], x["DestinationLat"]]]), axis=1)



df_ports["TotalExportDemand"] = df_demands[["Origin", "FFEPerWeek"]].groupby("Origin").sum()["FFEPerWeek"]
df_ports["TotalImportDemand"] = df_demands[["Destination", "FFEPerWeek"]].groupby("Destination").sum()["FFEPerWeek"]


ports_map = gpd.GeoDataFrame(df_ports, geometry=gpd.points_from_xy(df_ports["Longitude"], df_ports["Latitude"]))
demands_map = gpd.GeoDataFrame(df_demands)
dist_map = gpd.GeoDataFrame(df_dist)

df_fleet = df_fleet.join(df_fleet_count.set_index("Vessel class"), on="Vessel class")

Fleets data

In [3]:
df_fleet

,Vessel class,Capacity FFE,TC rate daily (fixed Cost),draft,minSpeed,maxSpeed,designSpeed,Bunker ton per day at designSpeed,Idle Consumption ton/day,panamaFee,suezFee,Quantity
0,Feeder_450,450,5000,8.0,10,14,12.0,18.8,2.4,64800.0,175769,8.0
1,Feeder_800,800,8000,9.5,10,17,14.0,23.7,2.5,115200.0,218445,8.0
2,Panamax_1200,1200,11000,12.0,12,19,18.0,52.5,4.0,172800.0,267217,4.0
3,Panamax_2400,2400,21000,11.0,12,22,16.0,57.4,5.3,345600.0,413533,NaN
4,Post_panamax,4200,35000,13.0,12,23,16.5,82.2,7.4,NaN,633007,NaN
5,Super_panamax,7500,55000,12.5,12,22,17.0,126.9,10.0,NaN,1035376,NaN


Ports data:

In [4]:
df_ports.head()

,name,Country,Cabotage_Region,D_Region,Longitude,Latitude,Draft,CostPerFULL,CostPerFULLTrnsf,PortCallCostFixed,PortCallCostPerFFE,TotalExportDemand,TotalImportDemand
UNLocode,,,,,,,,,,,,,
MAAGA,Agadir,Morocco,Morocco,West Med,-9.6670,30.5000,9.5,70.0,111.0,6274.0,5.0,130.0,148.0
EGALY,Alexandria,Egypt,Egypt,West Med,29.8906,31.1875,9.5,31.0,109.0,2021.0,14.0,214.0,487.0
ESALG,Algeciras,Spain,Spain,West Med,-5.4500,36.1250,13.5,229.0,136.0,773.0,11.0,1322.0,524.0
DZALG,Algiers,Algeria,Algeria,West Med,3.0468,36.7812,8.0,147.0,105.0,9937.0,3.0,10.0,231.0
TRAMB,Ambarli,Turkey,Turkey,West Med,28.7450,40.9830,9.5,146.0,19.0,9517.0,19.0,356.0,425.0


Demands data:

In [5]:
df_demands.head()

,Origin,Destination,FFEPerWeek,Revenue_1,TransitTime,DestinationLat,DestinationLong,OriginLat,OriginLong,distance,geometry
0,ESALG,TRAMB,266,330,14,40.983000,28.745000,36.12500,-5.45000,1811,"LINESTRING (-5.45 36.125, 28.745 40.983)"
1,ESALG,EGALY,190,350,6,31.187500,29.890600,36.12500,-5.45000,1819,"LINESTRING (-5.45 36.125, 29.8906 31.1875)"
2,ESALG,MACAS,180,350,5,33.583333,-7.616666,36.12500,-5.45000,191,"LINESTRING (-5.45 36.125, -7.616666 33.583333)"
3,EGPSD,MACAS,148,340,14,33.583333,-7.616666,31.21583,32.35722,2132,"LINESTRING (32.35722 31.21583, -7.616666 33.58..."
4,ITGOA,MAPTM,146,740,19,35.885000,-5.484700,44.42000,8.95000,878,"LINESTRING (8.95 44.42, -5.4847 35.885)"


Distances data:

In [6]:
df_dist.head()

,fromUNLOCODE,toUNLOCODE,Distance,Draft,IsPanama,IsSuez,DestinationLat,DestinationLong,OriginLat,OriginLong,geometry
4092,BGVAR,CYLMS,870,NaN,0,0,34.66667,33.03333,43.216666,27.916666,"LINESTRING (27.916666 43.216666, 33.03333 34.6..."
4098,BGVAR,DZAAE,1315,NaN,0,0,36.89060,7.76560,43.216666,27.916666,"LINESTRING (27.916666 43.216666, 7.7656 36.8906)"
4099,BGVAR,DZALG,1532,NaN,0,0,36.78120,3.04680,43.216666,27.916666,"LINESTRING (27.916666 43.216666, 3.0468 36.7812)"
4100,BGVAR,DZBJA,1431,NaN,0,0,36.75000,5.08333,43.216666,27.916666,"LINESTRING (27.916666 43.216666, 5.08333 36.75)"
4101,BGVAR,DZORN,1730,NaN,0,0,35.68750,-0.62500,43.216666,27.916666,"LINESTRING (27.916666 43.216666, -0.625 35.6875)"


Visualise demands, revenues, and distances

In [7]:

base = folium.Map(tiles="CartoDB Positron", location=[df_ports["Latitude"].mean(), df_ports["Longitude"].mean()], min_zoom=4, zoom_start=4, max_bounds=True, min_lat=df_ports["Latitude"].min() - 10, max_lat=df_ports["Latitude"].max() + 10, min_lon=df_ports["Longitude"].min() - 10, max_lon=df_ports["Longitude"].max() + 10)
ports_map.explore(m=base, name="Ports", marker_type="marker", marker_kwds=dict(icon=folium.map.Icon()))
ports_map.explore(m=base, name="TotalExportDemand", scheme="naturalbreaks", column="TotalExportDemand", marker_kwds=dict(radius=10, fill=True), k = 5, cmap="viridis")
ports_map.explore(m=base, name="TotalImportDemand", scheme="naturalbreaks", column="TotalImportDemand", marker_kwds=dict(radius=10, fill=True), k = 5, cmap="viridis")

g_demand = folium.FeatureGroup(name="Demand")
g_revenue = folium.FeatureGroup(name="Revenue")
g_distance = folium.FeatureGroup(name="Distance")
base.add_child(g_demand)
base.add_child(g_revenue)
base.add_child(g_distance)

for i, row in df_ports.iterrows():
    if row["TotalExportDemand"] > 0:
        demands_map[demands_map["Origin"] == row.name].explore(m=g_demand, scheme="naturalbreaks", column="FFEPerWeek", cmap="viridis", tags=["Demand", row.name], legend=False, show=True)
        demands_map[demands_map["Origin"] == row.name].explore(m=g_revenue, scheme="naturalbreaks", column="Revenue_1", cmap="viridis", tags=["Revenue", row.name], legend=False, show=True)
        demands_map[demands_map["Origin"] == row.name].explore(m=g_distance, scheme="naturalbreaks", column="distance", cmap="viridis", tags=["Distance",row.name], legend=False, show=True)


# folium.plugins.TagFilterButton(["Demand", "Revenue", "Distance"]).add_to(base)
folium.plugins.TagFilterButton(df_ports.sort_values("TotalExportDemand", ascending=False).index.unique().to_list(), open_popup_on_hover=True).add_to(base)

folium.LayerControl(collapsed=False).add_to(base)


base

/home/tatdatnguyen/python-projects/lsndp-viz/.venv/lib/python3.11/site-packages/mapclassify/classifiers.py:687: UserWarning: Not enough unique values in array to form 5 classes. Setting k to 3.
  self._classify()
/home/tatdatnguyen/python-projects/lsndp-viz/.venv/lib/python3.11/site-packages/mapclassify/classifiers.py:687: UserWarning: Not enough unique values in array to form 5 classes. Setting k to 3.
  self._classify()
/home/tatdatnguyen/python-projects/lsndp-viz/.venv/lib/python3.11/site-packages/mapclassify/classifiers.py:687: UserWarning: Not enough unique values in array to form 5 classes. Setting k to 3.
  self._classify()
/home/tatdatnguyen/python-projects/lsndp-viz/.venv/lib/python3.11/site-packages/mapclassify/classifiers.py:687: UserWarning: Not enough unique values in array to form 5 classes. Setting k to 4.
  self._classify()
/home/tatdatnguyen/python-projects/lsndp-viz/.venv/lib/python3.11/site-packages/mapclassify/classifiers.py:687: UserWarning: Not enough unique value

Read a sample solution (by BrouerDesaulniersPisinger2014)

In [8]:
solution = []

with open("sample_solution/Med_base_best.log", "r") as f:
    lines = f.readlines()
    start = False
    for line in lines:
        if start:
            if not line.split():
                start = False
                polygon = shapely.geometry.Polygon(polygon)
                solution.append((polygon, n_vessels))
                
            else:
                if line.startswith(" Butterfly rotation "):
                    continue
                port = line.split()[1]
                polygon.append((df_ports.loc[port, "Longitude"], df_ports.loc[port, "Latitude"]))
                
                
        if line.startswith(" # vessels"):
            start = True
            polygon = []
            n_vessels = int(line.split()[-1].strip())

solution_map = gpd.GeoDataFrame(solution, columns=["geometry", "n_vessels"])
solution_map.head()




,geometry,n_vessels
0,"POLYGON ((-7.61667 33.58333, -9.667 30.5, -8.7...",4
1,"POLYGON ((35.79 35.52, 34.9687 32.7968, 31.796...",1
2,"POLYGON ((-4.42747 36.71815, 3.0468 36.7812, 6...",3
3,"POLYGON ((10.1718 36.7968, 15.9 38.41667, 13.7...",3
4,"POLYGON ((35.5 33.8593, 32.35722 31.21583, 15....",4


Visualise the sample solution

In [9]:
base = folium.Map(tiles="CartoDB Positron", location=[df_ports["Latitude"].mean(), df_ports["Longitude"].mean()], min_zoom=4, zoom_start=4, max_bounds=True, min_lat=df_ports["Latitude"].min() - 10, max_lat=df_ports["Latitude"].max() + 10, min_lon=df_ports["Longitude"].min() - 10, max_lon=df_ports["Longitude"].max() + 10)
ports_map.explore(m=base, name="Ports", marker_type="marker", marker_kwds=dict(icon=folium.map.Icon()))

colours = ["#FF5733", "#33FF57", "#3357FF", "#FF33A6", "#FFC300", "#33FFF0", "#8C33FF"]

for i, row in solution_map.iterrows():
    folium.GeoJson(row["geometry"], name=f"route_{i}_{row['n_vessels']}vessels", color=colours[i], fillOpacity=0, popup=folium.Popup(str(f"n_vessels: {row['n_vessels']}"))).add_to(base)


folium.LayerControl(collapsed=False).add_to(base)

base